# Genetic algorithms

Genetic algorithms are one of the tools we can use to apply machine learning to finding good, sometimes even optimal, solutions to problems that have billions of potential solutions. They use biological processes in software to find answers to problems that have really large search spaces by continuously generating candidate solutions, evaluating how well the solutions fit the desired outcome, and refining the best solutions.

Based on Genetic Algorithms with Python by Clinton Sheppard.

The final code from this book is available at [github](https://github.com/handcraftsman/GeneticAlgorithmsWithPython).


## Guess the Password

In [1]:
import random
import statistics
import time
import sys
import unittest
import datetime

#### Generate a guess 
Next the algorithm needs a way to generate a random string from the gene set.

> random.sample takes sampleSize values from the input without replacement. This means there will be no duplicates in the generated parent unless geneSet contains duplicates, or length is greater than len(geneSet). The implementation above can generate a long string with a small set of genes and uses as many unique genes as possible.



In [2]:
def _generate_parent(length, geneSet, get_fitness):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    genes = ''.join(genes)
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness)

In [3]:
def _mutate(parent, geneSet, get_fitness):
    index = random.randrange(0, len(parent.Genes))
    childGenes = list(parent.Genes)
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate \
        if newGene == childGenes[index] \
        else newGene
    genes = ''.join(childGenes)
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness)

In [4]:
def get_best(get_fitness, targetLen, optimalFitness, geneSet, display):
    random.seed()
    bestParent = _generate_parent(targetLen, geneSet, get_fitness)
    display(bestParent)
    if bestParent.Fitness >= optimalFitness:
        return bestParent
    while True:
        child = _mutate(bestParent, geneSet, get_fitness)
        if bestParent.Fitness >= child.Fitness:
            continue
        display(child)
        if child.Fitness >= optimalFitness:
            return child
        bestParent = child

In [5]:
class Chromosome:
    Genes = None
    Fitness = None

    def __init__(self, genes, fitness):
        self.Genes = genes
        self.Fitness = fitness

In [6]:
class Benchmark:
    @staticmethod
    def run(function):
        timings = []
        stdout = sys.stdout
        for i in range(100):
            sys.stdout = None
            startTime = time.time()
            function()
            seconds = time.time() - startTime
            sys.stdout = stdout
            timings.append(seconds)
            mean = statistics.mean(timings)
            if i < 10 or i % 10 == 9:
                print("{0} {1:3.2f} {2:3.2f}".format(
                    1 + i, mean,
                    statistics.stdev(timings, mean)
                    if i > 1 else 0))

#### Fitness
The fitness value the genetic algorithm provides is the only feedback the engine gets to guide it toward a solution. In this project the fitness value is the total number of letters in the guess that match the letter in the same position of the password.

In [7]:
def get_fitness(guess, target):
    return sum(1 for expected, actual in zip(target, guess)
               if expected == actual)

In [8]:
def display(candidate, startTime):
    timeDiff = datetime.datetime.now() - startTime
    print("{0}\t{1}\t{2}".format(
        candidate.Genes, candidate.Fitness, str(timeDiff)))

####  Genes 
To begin with, the genetic algorithm needs a gene set to use for building guesses. For this project that will be a generic set of letters.

In [9]:
geneset = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!.,"

def guess_password(target):
    startTime = datetime.datetime.now()

    def fnGetFitness(genes):
        return get_fitness(genes, target)

    def fnDisplay(candidate):
        display(candidate, startTime)

    optimalFitness = len(target)
    best = get_best(fnGetFitness, len(target),
                            optimalFitness, geneset, fnDisplay)
    if best.Genes == target:
        print("é igual!")
    else:
        print("é diferente!")


It also needs a target password to guess. Below you have a senteces to test.

In [10]:
target = "Hello World!"

guess_password(target)

czqjBgEkf,mQ	0	0:00:00
czqjBgWkf,mQ	1	0:00:00.002000
ceqjBgWkf,mQ	2	0:00:00.002999
ceqjBgWkf,m!	3	0:00:00.005998
ceqjBgWkr,m!	4	0:00:00.007002
celjBgWkr,m!	5	0:00:00.008032
celjBgWor,m!	6	0:00:00.008032
celjBgWorlm!	7	0:00:00.013030
HeljBgWorlm!	8	0:00:00.017001
HeljB Worlm!	9	0:00:00.017001
Heljo Worlm!	10	0:00:00.018048
Heljo World!	11	0:00:00.019032
Hello World!	12	0:00:00.020030
é igual!


In [11]:
target = "For I am fearfully and wonderfully made."

guess_password(target)

qCEfKGVHLjogW!n.,sTdwr mPQUpMYARbhZtNOzF	1	0:00:00
qCEfKGVHLjogW!n.,sTdwr mPQUpMfARbhZtNOzF	2	0:00:00.001011
qCEfKGVHLjogW!n.,sTdwd mPQUpMfARbhZtNOzF	3	0:00:00.002002
qCEfKGVHLjogW!n.,sTdwd mPQUpMfAlbhZtNOzF	4	0:00:00.003002
qCEfKGVHLjogW!n.,sTdwd mPQUpMfAlbhZtaOzF	5	0:00:00.004004
qCEfKGVHLjogW!n.,sTawd mPQUpMfAlbhZtaOzF	6	0:00:00.004004
qCEfIGVHLjogW!n.,sTawd mPQUpMfAlbhZtaOzF	7	0:00:00.005001
qCEfIGVHLfogW!n.,sTawd mPQUpMfAlbhZtaOzF	8	0:00:00.006006
qCEfIGVHLfogW!n.,sTawd mPnUpMfAlbhZtaOzF	9	0:00:00.009029
qCEfIGVHLfogW!nl,sTawd mPnUpMfAlbhZtaOzF	10	0:00:00.009029
qCEfIGVHLfogW!nl,sTawd wPnUpMfAlbhZtaOzF	11	0:00:00.011034
qoEfIGVHLfogW!nl,sTawd wPnUpMfAlbhZtaOzF	12	0:00:00.014036
qoEfIGVHLfogW!nllsTawd wPnUpMfAlbhZtaOzF	13	0:00:00.014036
qoEfIGVHLfogW!nllsTawd wPnUpMfAlbhZtaOz.	14	0:00:00.019001
qoEfIGVmLfogW!nllsTawd wPnUpMfAlbhZtaOz.	15	0:00:00.021001
qoEfIGVmLfogWfnllsTawd wPnUpMfAlbhZtaOz.	16	0:00:00.022003
qoEfIGVmLfogWfnllsTawd wPnUpMfAlbhZmaOz.	17	0:00:00.022003
qoEfIGVmLfogW

In [12]:
target = "To be or not to be."

guess_password(target)

kvLCXpceFxObfVos!ZS	1	0:00:00
kvLCXpceFxObfVo !ZS	2	0:00:00.001001
kvLCXpceFxObfVo !Z.	3	0:00:00.004000
kvLbXpceFxObfVo !Z.	4	0:00:00.004997
koLbXpceFxObfVo !Z.	5	0:00:00.005997
koLbXpceFxobfVo !Z.	6	0:00:00.005997
koLbepceFxobfVo !Z.	7	0:00:00.007000
koLbepceFxobfto !Z.	8	0:00:00.007998
koLbepcrFxobfto !Z.	9	0:00:00.021006
koLbepcrFxob to !Z.	10	0:00:00.024017
koLbepcrFxob to !e.	11	0:00:00.025999
koLbe crFxob to !e.	12	0:00:00.025999
ToLbe crFxob to !e.	13	0:00:00.026998
ToLbe cr xob to !e.	14	0:00:00.029998
To be cr xob to !e.	15	0:00:00.032996
To be cr xob to be.	16	0:00:00.033997
To be cr xot to be.	17	0:00:00.042027
To be or xot to be.	18	0:00:00.045027
To be or not to be.	19	0:00:00.053029
é igual!


In [13]:
length = 50
target = ''.join(random.choice(geneset) for _ in range(length))
print(target+"\n\n")
guess_password(target)

.hOQXaQbVdDGHqTBmSzt.SCMQyZOZYQlsgOJfqtBQSj RcuRHR


selwiYqzXGmcOhjUL.uNEfVxCBbP tknTQvRFgWpoADdraS!KZ	0	0:00:00
selwiYqzXGmcOhjUL.uNEfVxQBbP tknTQvRFgWpoADdraS!KZ	1	0:00:00.000998
selwiYqzXGmcOhjUL.uNESVxQBbP tknTQvRFgWpoADdraS!KZ	2	0:00:00.001999
selwiYqzXGmcOhjUL.uNESVxQBbP tknTQvRFgWpoAD raS!KZ	3	0:00:00.001999
selwiYqzXGmcOhjUL.uNESVxQBbP tknTQvJFgWpoAD raS!KZ	4	0:00:00.005998
selwiYqzXGmcOhjUL.uNESVxQBbP tknTQvJFgWpoAD rcS!KZ	5	0:00:00.005998
selwiYqzXGmcOhjUL.uNESVxQBbP tknTQvJFgWpoSD rcS!KZ	6	0:00:00.005998
selwiYqzXGmcOhjUL.uNESVxQBbO tknTQvJFgWpoSD rcS!KZ	7	0:00:00.007996
selwiYqzXGmcOhjUL.zNESVxQBbO tknTQvJFgWpoSD rcS!KZ	8	0:00:00.007996
selwiYqzXGDcOhjUL.zNESVxQBbO tknTQvJFgWpoSD rcS!KZ	9	0:00:00.007996
selwiYqzXGDcOhjUL.zNESVxQBbO tknTQvJFgWpoSj rcS!KZ	10	0:00:00.009000
selwiYqzXGDcOhjUm.zNESVxQBbO tknTQvJFgWpoSj rcS!KZ	11	0:00:00.009000
selwiYqbXGDcOhjUm.zNESVxQBbO tknTQvJFgWpoSj rcS!KZ	12	0:00:00.010999
selwiYqbXGDcOhjUm.zNESVxQBbO tknTQvJFgWpoSj rcSRKZ	13	0:00:00.01304

In [ ]:
Benchmark.run(lambda: test_Random())